# Backmap a MARTINI CG structure
To obtain an atomistic structure from coarse grained simulations we need a CG structure and a topology file for the atomistic system.

In [1]:
import sys
sys.path.insert(0, '../..')
import backward
import running_rabbit as rr

In [2]:
# input
cg_structure = "2xUBQ_cg.pdb"            # coarse grained structure for backmapping
aa_topology  = "2xUBQ_aa.top"            # atomistic topology

# output
aa_structure_raw = "2xUBQ_aa_raw.gro"    # initial structure after running backward.py
aa_structure     = "2xUBQ_aa.gro"        # final structure after energy min.
path_em          = "./em_after_backmap"  # dir for running rabbit

## Backmapping

In [3]:
args = '-f {} -p {} -o {} -to gromos -kick 0.05'.format(cg_structure, aa_topology, aa_structure_raw)
args_list = args.split()
backward.workflow(args_list)

Error reading charmm27 to martini mapping for POPG (file: ../../Mapping/pops.amber.map).
Error reading charmm36 to martini mapping for POPG (file: ../../Mapping/pops.amber.map).
Error reading slipids to martini mapping for POPG (file: ../../Mapping/pops.amber.map).
Error reading amber to martini mapping for POPG (file: ../../Mapping/pops.amber.map).
Error reading charmm27 to martini mapping for DOPE (file: ../../Mapping/dopg.charmm36.map).
Error reading charmm36 to martini mapping for DOPE (file: ../../Mapping/dopg.charmm36.map).
Error reading slipids to martini mapping for DOPE (file: ../../Mapping/dopg.charmm36.map).
Error reading amber to martini mapping for DOPE (file: ../../Mapping/dopg.charmm36.map).
Error reading charmm27 to martini mapping for DOPG (file: ../../Mapping/dops.charmm36.map).
Error reading charmm36 to martini mapping for DOPG (file: ../../Mapping/dops.charmm36.map).
Error reading slipids to martini mapping for DOPG (file: ../../Mapping/dops.charmm36.map).
Error rea

---
The atomistic structure after backmapping is highly unphysical and has to be energy minimized to obtain a proper structure. Therefore, we perform energy minimization in three steps:
1. Energy minimization with l-bfgs 
2. Backbone position restrained, bigger em step, lower force threshold, no bond constraints
3. no position restraints, bigger em step, lower force threshold, bonds constrained with LINKS 
---

## Running rabbit

Running rabbit performes simulation routines defined in templates for reproducable energy minimization

In [5]:
rabbit              = rr.Rabbit(ff="gromos54a7", template_name="backmap_em_bfgs")
rabbit.structure    = aa_structure_raw
rabbit.topology     = aa_topology
rabbit.destination  = path_em
# make all necessary run files like .mdp
rabbit.run()

True

In [6]:
# run energy minimization and equilibration
!cd {path_em} && bash setup

In [7]:
# copy final structure
!cp {path_em}/eq5.gro {aa_structure}

In [8]:
# note: some atoms are not in the first unit cell after mackmapping
# resolve by:
!gmx editconf -f {aa_structure} -o {aa_structure} -pbc yes

                      :-) GROMACS - gmx editconf, 2020 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     